In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q tensorflow
!pip install -q tensorflow_ranking

     |████████████████████████████████| 462 kB 7.5 MB/s 
     |████████████████████████████████| 141 kB 7.4 MB/s 


In [102]:
import numpy as np
import pandas as pd
import datetime as dt
import os

PATH = os.getcwd()
#PATH = '/content/drive/Shareddrives/Master Tesis/Tesis'

In [103]:
import tensorflow_ranking as tfr
import tensorflow as tf
from tensorflow_serving.apis import input_pb2

## Import Tables

In [111]:
tables_names = ['big_log_ret','big_RCV', 'big_RVT', 'big_positivePartscr', 'big_negativePartscr', 'big_splogscr', 'big_linscr', 'big_linscr','big_lag1_log_ret','big_lag4_log_ret','big_lag1_month_log_ret']
tables_dict = {}
for name in tables_names:
    table = pd.read_csv(os.path.join(PATH, 'Tables', name+'.csv'))
    table['Date'] = pd.to_datetime(table['Date']).dt.date
    table.set_index('Date', inplace=True)
    tables_dict[name] = table

tables_dict['big_RCV'].head()

,AAL,AAPL,ABBV,ABC,ABT,ADP,AIG,AMD,AMZN,AXP,...,UAL,UNH,UPS,USB,V,VZ,WFC,WMT,WY,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-08,8.4760,13.162167,NaN,0.404000,-24.607200,NaN,NaN,NaN,-3.370000,NaN,...,12.493714,NaN,-9.7618,8.849167,NaN,8.031400,7.316500,33.678600,NaN,1.957143
2012-01-15,NaN,11.965000,NaN,-4.760143,-3.706000,NaN,-11.584333,16.834600,-18.815571,-10.443167,...,12.935143,11.435500,-1.2800,-17.410857,-33.085800,-10.427667,21.078333,15.656600,NaN,-18.020000
2012-01-22,9.9768,-10.776667,NaN,-17.240429,-22.233833,NaN,-10.898400,NaN,15.001167,0.860833,...,2.058714,17.216800,-8.6162,-24.026800,NaN,-31.776000,-0.758500,-14.170714,NaN,-19.522714
2012-01-29,-20.9400,-9.334000,NaN,11.740000,-8.165800,-42.6148,-22.226000,NaN,-7.894167,-33.329167,...,9.101000,NaN,22.8662,-52.884000,NaN,8.221000,-20.201500,0.036800,NaN,6.625571
2012-02-05,NaN,-33.438857,NaN,-2.313000,-30.020400,-39.8710,-18.959800,-30.327667,-3.668714,-20.110167,...,-5.529429,-36.037833,4.4648,-42.687167,14.249667,-47.767143,-7.948200,-45.108167,-28.786,8.348429


## Merging Tables - Building the query-stock final table

In [112]:
final_table = tables_dict['big_log_ret'].stack() #Stack DF to a Series so that is grouped by Date and then by Ticker
final_table.index.names=('Query','Ticker') #Rename 'Date' to 'Query'
final_table = final_table.reset_index()
final_table.rename(columns={0: 'big_log_ret'}, inplace=True)
resting_tables_dict = dict(tables_dict) #Make a copy of the dictionary
del resting_tables_dict['big_log_ret'] #Remove the log_ret table

for i, t_name in enumerate(resting_tables_dict):
    table = tables_dict[t_name].stack()
    table.index.names=('Query','Ticker')
    table = table.reset_index()
    table.rename(columns={0: t_name}, inplace=True)
    final_table = pd.merge(final_table, table, on=['Query','Ticker'])

final_table.set_index('Query', inplace=False).to_csv(os.path.join(PATH,'Tables','final_table.csv')) #Store in csv format in the 'Tables' folder
final_table

,Query,Ticker,big_log_ret,big_RCV,big_RVT,big_positivePartscr,big_negativePartscr,big_splogscr,big_linscr,big_lag1_log_ret,big_lag4_log_ret,big_lag1_month_log_ret
0,2012-01-08,AAL,0.063980,8.476000,0.000280,0.020240,0.013340,0.010760,54.733580,0.099426,0.094986,-0.003565
1,2012-01-08,AAPL,-0.006150,13.162167,0.003400,0.016050,0.015650,0.020333,48.380133,0.042066,-0.032534,0.070567
2,2012-01-08,ABC,-0.020683,0.404000,0.000760,0.027820,0.012980,0.017000,65.547120,0.036953,0.010394,0.059249
3,2012-01-08,ABT,0.000864,-24.607200,0.000320,0.010820,0.018540,-0.012980,27.375300,-0.006602,0.005847,0.023364
4,2012-01-08,AMZN,-0.023212,-3.370000,0.001100,0.016180,0.020000,-0.024640,40.688020,0.053483,-0.062913,-0.055493
...,...,...,...,...,...,...,...,...,...,...,...,...
42050,2021-11-28,UPS,-0.022512,-42.160333,0.000317,0.018200,0.024717,-0.008817,46.677617,-0.015323,-0.024901,-0.034095
42051,2021-11-28,USB,-0.033782,-28.514833,0.000483,0.020517,0.010367,0.037033,68.229333,-0.001564,0.005287,-0.048707
42052,2021-11-28,VZ,-0.007363,-22.016000,0.000686,0.018957,0.036629,-0.066443,28.958114,0.018313,-0.014255,-0.022713
42053,2021-11-28,WFC,-0.014140,-37.779000,0.000983,0.004283,0.017333,-0.061550,15.077883,-0.008021,-0.010320,-0.050985


In [106]:
'''
Aqui tenemos q añadir un filtro para eliminar aquellas filas q tengan un NaN y tambien si queremos hacer algun preprocessing previo
'''
final_table.isnull().values.any()

False

In [12]:
'''
Es posible que necesitemos pasar todos los log returns a no negativos sumandole el valor absoluto del minimo a toda la columna para que sean positivos
'''

False

## Data Splitting

We are looking to split the data to train our model, the problem with time series split train tests is that we use 95% to train the data and test it on the last 5%. There is a possibility that our test data behaves nicely or we just got lucky. To combat this event we do an expanding window test. Where we use some of the data to predict the next period, then we expand the data and repeat. This way we can have a more testing sets with the same data in an appropiate way.  

In [113]:
def expanding_window_split(dataframe, number_of_splits:int):
    '''
    Splits the dataframe into 'number of splits' to train and test sets to predict one period ahead. It produces the number of splits starting from the
    last element onwards in a decreasing manner. 
    '''
    all_splits = {}
    for i in range(number_of_splits):
        test_query = dataframe['Query'].unique()[-(i+1)]
        train_query = dataframe['Query'].unique()[:-(i+1)]
        test = dataframe[dataframe['Query'].isin([test_query])]
        train = dataframe[dataframe['Query'].isin(train_query)]
        all_splits["split_"+str(number_of_splits-i)] = (train, test)
    return all_splits

In [176]:
'''
Protobuffers are extensible structures suitable for storing data in a serialized format, either locally or in a distributed manner.
TF ranking has a couple of pre-defined protobufs such as ELWC which make it easier to integrate and formalize data ingestion into
the ranking pipeline.

Protocol buffers and the tf.data API is a set of utilities that provide a mechanism to read and store data for efficient loading
and preprocessing in a way that's fast and scalable.
'''

# Class I wrote to organize code resposible for parsing our dataframe data and 
# creating compressed TF-Records in ELWC protobuf format so that they used by 
# most rankers in TF-Ranking and be compatible with future rankers or new methods

class DFToELWCProto():
    """ Class to parse Dataframe ranking data in ELWC proto TFRecords"""
    
    def __init__(self, dir:str=".", use_compression:bool=False):
        assert isinstance(dir,str)
        if not os.path.isdir(dir):
            os.mkdir(dir)
        self.input_path = dir
        assert isinstance(use_compression,bool)
        self.compress = use_compression
        if self.compress:
            self.compress_type = 'GZIP'
        else:
            self.compress_type = None


    # Helper functions (see also https://www.tensorflow.org/tutorials/load_data/tf_records)
    def _bytes_feature(self,value_list):
        """Returns a bytes_list from a string / byte."""
        if isinstance(value_list, type(tf.constant(0))):
            value_list = value_list.numpy()
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value_list]))


    def _float_feature(self,value_list):
        """Returns a float_list from a float / double."""
        return tf.train.Feature(float_list=tf.train.FloatList(value=[value_list]))

    def _int64_feature(self,value_list):
        """Returns an int64_list from a bool / enum / int / uint."""
        return tf.train.Feature(int64_list=tf.train.Int64List(value=[value_list]))


    def print_topn_tfrecord(self, target_filename, num_of_examples_to_print=-1):
        filenames = [target_filename]
        tf_record_dataset = tf.data.TFRecordDataset(filenames,
                                                    compression_type=self.compress_type)

        for raw_record in tf_record_dataset.take(num_of_examples_to_print):
            example_list_with_context = input_pb2.ExampleListWithContext()
            example_list_with_context.ParseFromString(raw_record.numpy())
            print(example_list_with_context)
    
    def read_topn_tfrecord(self, target_filename, num_of_examples_to_read=-1):
        filenames = [target_filename]
        tf_record_dataset = tf.data.TFRecordDataset(filenames,
                                                    compression_type=self.compress_type)
        queries_with_context_and_exmples=[]
        for raw_record in tf_record_dataset.take(num_of_examples_to_read):
            example_list_with_context = input_pb2.ExampleListWithContext()
            example_list_with_context.ParseFromString(raw_record.numpy())
            queries_with_context_and_exmples.append(example_list_with_context)
        return queries_with_context_and_exmples

    def df_to_TFrecord(self, df, file_name):

        """ 
        for reading and converting directly from arrays in memory
        
        """    
        file_name = os.path.basename(file_name)

        if self.compress:
            print('Using GZIP compression for writing ELWC TFRecord Dataset')
            opts = tf.io.TFRecordOptions(compression_type = self.compress_type)
            file_name = f"{file_name}.gzipped_tfrecord"
        else:
            opts = tf.io.TFRecordOptions(compression_type = self.compress_type)
            file_name = f"{file_name}.tfrecord"
        save_path = f"{self.input_path}/{file_name}"

        with tf.io.TFRecordWriter(f"{save_path}", options=opts) as writer:
            
            input_array = np.array(df)
            col_names = df.columns

            ELWC = input_pb2.ExampleListWithContext()
            prev_qid = None

            for i in range(input_array.shape[0]):

                qid, doc, r, features = str(input_array[i,0]), input_array[i,1], input_array[i,2], input_array[i,3:]

                example_proto_dict = {
                              f"{col_names[f_n+3]}":self._float_feature((f_v))
                                  for (f_n, f_v) in enumerate(features)
                          }
                example_proto_dict['rel'] = self._float_feature(r)
                example_proto_dict['doc'] = self._bytes_feature(doc.encode('utf-8'))

                example_proto = tf.train.Example(features=tf.train.Features(feature=example_proto_dict))
                if qid != prev_qid:
                    if prev_qid is not None:
                        writer.write(ELWC.SerializeToString())
                    prev_qid = qid
                    
                    context_proto_dict = {
                        "qid" : self._bytes_feature("{}".format(qid).encode('utf-8'))
                        }
                    context_proto = tf.train.Example(features=tf.train.Features(feature=context_proto_dict))
                    
                    ELWC = input_pb2.ExampleListWithContext()
                    ELWC.context.CopyFrom(context_proto)
                    ELWC.examples.append(example_proto)
                else:
                    ELWC.examples.append(example_proto)

            # final write for the last query grp
            writer.write(ELWC.SerializeToString())

In [173]:
from sklearn.model_selection import train_test_split

#Get the unique Queries and do the split based on that
query_train, query_test = train_test_split(final_table.Query.unique(), test_size=0.3, random_state=42, shuffle=False)
train = final_table[final_table.Query.isin(query_train)]
test = final_table[final_table.Query.isin(query_test)]

#Convert Train and Test dataset to ELWC and store it
ELWC_converter = DFToELWCProto(dir = "./Dataset-tfrecords/70v30Split", use_compression=False)
ELWC_converter.df_to_TFrecord(train,"train")
ELWC_converter.df_to_TFrecord(test,"test")

In [116]:
#Number of splits that we will do to our dataframe 
_NUM_SPLITS = 48

#Create a dictionary of tuples containing all the splits we will use on our LTR model. Each tuple contains two dataframes (train, test)
splits_dict = expanding_window_split(final_table, _NUM_SPLITS)

#Convert all those dataframes into ELWC
for i in range(_NUM_SPLITS):
    split_name = "split_{}".format(i+1)
    train, test = splits_dict[split_name][0], splits_dict[split_name][1]
    
    #Convert Train dataset to ELWC and store it
    ELWC_converter_train = DFToELWCProto(dir = "./Dataset-tfrecords/Split_{}".format(i+1), use_compression=False)
    ELWC_converter_train.df_to_TFrecord(train,"train")
    
    #Convert Test dataset to ELWC and store it
    ELWC_converter_test = DFToELWCProto(dir = "./Dataset-tfrecords/Split_{}".format(i+1), use_compression=False)
    ELWC_converter_test.df_to_TFrecord(test,"test")